### Read data 

In [1]:
from datasets import load_dataset
data = load_dataset("uit-nlp/vietnamese_students_feedback")

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 11426 examples [00:00, 49669.04 examples/s]
Generating validation split: 1583 examples [00:00, 34206.49 examples/s]
Generating test split: 3166 examples [00:00, 49037.17 examples/s]


In [3]:
data#['train']

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})

In [4]:
read_data(data)

NameError: name 'read_data' is not defined

In [5]:
data['train']['sentence'][12]

'đang dạy thầy wzjwz208 đi qua nước ngoài giữa chừng , thầy wzjwz209 dạy thay .'

In [6]:
train_data = data['train']#['train'][12]
#train_data['sentence']=pd.DataFrame(train_data['sentence']) 


In [ ]:
train_data

In [ ]:
data['train']['sentence'][12]

### 1. Lower case

In [7]:
def lower_case(text):
    return text.lower()
    
# Apply the lower_case function to the 'sentence' column
train_data = train_data.map(lambda example: {'sentence': lower_case(example['sentence'])})

Map: 100%|██████████| 11426/11426 [00:00<00:00, 25249.97 examples/s]


### 2. Transform emoticons to word form

In [8]:
from _2_0_dictionary import emotion2wordform_dict, wordform2vnese_dict, mispelling_dict, translate_dict, number_dict

def emoticon2word(sentence):
    words = sentence.split()
    converted_words = [emotion2wordform_dict.get(word, word) for word in words]
    converted_sentence = ' '.join(converted_words)
    return converted_sentence

# Test the function
sentence = "giao viên ko cóa dạy :)"
print("Converted sentence:", emoticon2word(sentence))

Converted sentence: giao viên ko cóa dạy colonsmile


### 3. Remove punctuation

In [9]:
import string

def remove_punctuation(input_string):
    # Create a translation table to remove all punctuation characters
    translator = str.maketrans('', '', string.punctuation)
    
    # Use the translate method to remove punctuation
    cleaned_string = input_string.translate(translator)
    
    return cleaned_string

print(remove_punctuation('thầy , cô dạy tốt . Trường ok.'))
#train_data = train_data.map(lambda example: {'sentence': remove_punctuation(example['sentence'])})

thầy  cô dạy tốt  Trường ok


### 4. Translate word form of emoticons into VNese

In [10]:
def word_form2Vnese(sentence):
    words = sentence.split()
    print(words)
    converted_words = [wordform2vnese_dict.get(word, word) for word in words]
    converted_sentence = ' '.join(converted_words)
    return converted_sentence

# Test the function
sentence = "giao_viên ko cóa dạy colonsmile"
print("Converted sentence:", word_form2Vnese(sentence))

['giao_viên', 'ko', 'cóa', 'dạy', 'colonsmile']
Converted sentence: giao_viên ko cóa dạy cười nhẹ


### 5. Transform number to text

In [11]:
# Test cùng với "chấm thầy 10 điểm"
def number2text(sentence):
    words = sentence.split()
    converted_words = [number_dict.get(word, word) for word in words]
    converted_sentence = ' '.join(converted_words)
    return converted_sentence

# Test the function
sentence = "chấm thầy 10 điểm" 
print("Converted sentence:", number2text(sentence))

Converted sentence: chấm thầy mười điểm


### 6. Spelling correction, acronyms

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

sentences = "giáo_viên" # nhận thấy token khác nhau, 
#test cùng với độ tự tin khi dự đoán, nếu input sai chính tả 
inputs = tokenizer(sentences, padding= True, truncation=True),
inputs

({'input_ids': [0, 960, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]},)

In [13]:
# Your dictionary tiếp cận 1 
# temp_dict = {
#     'không': ['ko', 'k', 'khum', 'khom', 'khom'],
#     'có': ['cóa', 'cóo', 'coá'],
#     'giảng viên': ['gv'], 
#     'giáo_viên' : ['gíao_viên', 'giao_viên']
# }

# def misspelling2true(sentence, dictionary):
#     words = sentence.split()
#     converted_words = [convert_list_in_dict(word, dictionary) for word in words]
#     converted_sentence = ' '.join(converted_words)
#     return converted_sentence

# def convert_list_in_dict(word, dictionary):
#     for key, values in dictionary.items():
#         if word in values:
#             return key
#     return word

# # Test the function with a sentence
# sentence_to_convert_back =  'giao viên này dạy ko hay như gv khác'
# converted_sentence_back = misspelling2true(sentence_to_convert_back, temp_dict)
# print("Converted sentence back:", converted_sentence_back)

In [14]:
#tiếp cận 2 
# dịch từ sai chính tả thành đúng chính tả
def mispell2word(sentence, dictionary = mispelling_dict):
    sentence = " " + sentence.strip() + " "
    for key, value_list in dictionary.items():
        for value in value_list:
            sentence = sentence.replace(value, key)
    return sentence

sentence = 'gv dạy hoc sinh trong phỏng máy k de hieu xíu nào'
print("Converted sentence back:", mispell2word(sentence))

Converted sentence back:  giảng viên dạy  học sinh  trong phòng máy không dễ hiểu xíu nào 


In [15]:
# dịch những từ tiếng anh thành tiếng việt 
def translate2word(sentence, dictionary = translate_dict):
    sentence = " " + sentence.strip() + " "
    for key, value_list in dictionary.items():
        for value in value_list:
            sentence = sentence.replace(value, key)
    return sentence

sentence = 'cô dạy max hay , rất pro , đưa bài lên web'
print("Converted sentence back:", translate2word(sentence))

Converted sentence back:  cô dạy cực kì hay , rất đỉnh , đưa bài lên trang mạng 


### 7. Remove extra whitespace

In [16]:
def remove_extra_whitespace(input_string):
    words = input_string.split()
    return ' '.join(words)

# Example usage
input_string = "This    is      a   string     with     extra     spaces."
print(remove_extra_whitespace(input_string))  # Output: "This is a string with extra spaces."

This is a string with extra spaces.


### 8. Remove number (digits)

In [17]:
train_data['sentence'][12]

'đang dạy thầy wzjwz208 đi qua nước ngoài giữa chừng , thầy wzjwz209 dạy thay .'

In [18]:
def remove_numbers(input_string):
    # Use the isalpha() method to filter out numeric characters
    cleaned_string = ''.join(char for char in input_string if not char.isdigit())
    return cleaned_string

train_data = train_data.map(lambda example: {'sentence': remove_numbers(example['sentence'])})

Map: 100%|██████████| 11426/11426 [00:00<00:00, 21278.54 examples/s]


In [19]:
train_data['sentence'][12]

'đang dạy thầy wzjwz đi qua nước ngoài giữa chừng , thầy wzjwz dạy thay .'

### 9. Word Segmentation

In [21]:
pip install underthesea

     ---------------------------------------- 0.0/139.2 kB ? eta -:--:--
     ---------- -------------------------- 41.0/139.2 kB 960.0 kB/s eta 0:00:01
     -------------------------------------- 139.2/139.2 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     -------- ------------------------------- 0.3/1.5 MB 7.0 MB/s eta 0:00:01
     ----------------------- ---------------- 0.9/1.5 MB 9.3 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/20.9 MB ? eta -:--:--
   -- ------------------------------------- 1.2/20.9 MB 37.3 MB/s eta 0:00:01
   ----- ---------------------------------- 2.8/20.9 MB 29.2 MB/s eta 0:00:01
   ------- -------------------------------- 4.2/20.9 MB 29.4 MB/s eta 0:00:01
   ---------- ----------------------------- 5.4/20.9 MB 28.6 MB/s eta 0:00:01
   ------------- -------------------------- 7.1/20.9 MB 30.5 MB/s eta 0:00:01
 

In [22]:
# thêm token bình thường
import underthesea
def word_segment(text):
    return underthesea.word_tokenize(text, format="text")

word_segment('gíao viên này dạy ko hay như gv khác')
#word_segment("Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây.")

'giáo_viên này dạy ko hay như gv khác'

In [23]:
train_data = train_data.map(lambda example: {'sentence': word_segment(example['sentence'])})

Map: 100%|██████████| 11426/11426 [00:08<00:00, 1355.40 examples/s]


In [24]:
train_data['sentence']

['slide giáo_trình đầy_đủ .',
 'nhiệt_tình giảng_dạy , gần_gũi với sinh_viên .',
 'đi học đầy_đủ full_điểm chuyên cần .',
 'chưa áp_dụng công_nghệ_thông_tin và các thiết_bị hỗ_trợ cho việc giảng_dạy .',
 'thầy giảng bài hay , có nhiều bài_tập ví_dụ ngay trên lớp .',
 'giảng_viên đảm_bảo thời_gian lên_lớp , tích_cực trả_lời câu hỏi của sinh_viên , thường_xuyên đặt câu hỏi cho sinh_viên .',
 'em sẽ nợ môn này , nhưng em sẽ học lại ở các học_kỳ kế_tiếp .',
 'thời_lượng học quá dài , không đảm_bảo tiếp_thu hiệu_quả .',
 'nội_dung môn_học có phần thiếu trọng_tâm , hầu_như là chung_chung , khái_quát khiến sinh_viên rất khó nắm được nội_dung môn_học .',
 'cần nói rõ hơn bằng cách trình_bày lên bảng thay_vì nhìn vào slide .',
 'thầy rất tận_tình và đi dạy rất đúng giờ .',
 'dễ bị áp_lực .',
 'đang dạy thầy wzjwz đi qua nước_ngoài giữa_chừng , thầy wzjwz dạy thay .',
 'môn_học còn dễ cần nâng_độ khó lên nữa .',
 'có_thể cho sinh_viên đi thăm_quan nhiều công_ty xem quy_mô và cách làm_việc , để g

### 10. Remove stopwords and particular words 

In [27]:
def remove_stopwords(input_text, stopwords_file='_2_stopword.txt'):
    # Read the custom stop words from the file
    with open(stopwords_file, 'r', encoding='utf-8') as file:
        stopwords = set(line.strip() for line in file)

    cleaned_words = [word for word in input_text.split() if word.lower() not in stopwords]
    cleaned_text = ' '.join(cleaned_words)

    return cleaned_text

# Test the function
input_text = "thầy rất tận tình và đi dạy rất đúng giờ"
cleaned_text = remove_stopwords(input_text, '_2_stopword.txt')
print(cleaned_text)

thầy tận tình đi dạy đúng giờ


In [28]:
train_data = train_data.map(lambda example: {'sentence': remove_stopwords(example['sentence'])})

Map: 100%|██████████| 11426/11426 [00:01<00:00, 6605.42 examples/s]


### 11. Key_clause extraction

In [29]:
import underthesea

def key_clause_extraction_vietnamese(text):
    sentences = underthesea.sent_tokenize(text)
    
    key_clauses = []
    for sentence in sentences:
        # Loại bỏ các câu không chứa thông tin (ví dụ: các câu chỉ chứa một số từ liên kết hoặc từ giới thiệu)
        if len(sentence.strip()) > 5:
            # Lưu lại câu chính đầu tiên (có thể mở rộng hơn để lựa chọn nhiều câu chính hơn)
            key_clauses.append(sentence)
            
    return key_clauses

# Ví dụ văn bản đầu vào (tiếng Việt)
text = "Trí tuệ nhân tạo (AI) là mô phỏng quá trình trí tuệ của con người bởi máy móc, đặc biệt là các hệ thống máy tính. Các quá trình này bao gồm học hỏi (tiếp thu thông tin và quy tắc để sử dụng thông tin đó), lập luận (sử dụng quy tắc để đạt đến kết luận gần đúng hoặc xác định) và tự sửa lỗi. AI có thể được phân loại thành AI yếu và AI mạnh. AI yếu, còn được gọi là AI hẹp, là một hệ thống AI được thiết kế và huấn luyện cho một nhiệm vụ cụ thể. Trợ lý cá nhân ảo, chẳng hạn như Siri của Apple, là một dạng AI yếu. AI mạnh, còn được gọi là trí tuệ nhân tạo tổng quát, là một hệ thống AI có khả năng tổng quát hóa như trí tuệ nhân tạo của con người. Khi được đưa ra một nhiệm vụ không quen thuộc, một hệ thống AI mạnh có thể tìm ra giải pháp mà không cần sự can thiệp của con người."
#text ="giảng viên phải nên cho ví dụ cụ thể những phần quan trọng nên nói kỹ có thể lấy ví dụ minh họa tránh việc lập đi lập lại nhiều lần sẽ tạo cảm giác chán cho sinh viên ."

# Trích xuất key clause
key_clauses = key_clause_extraction_vietnamese(text)

# In kết quả
for i, clause in enumerate(key_clauses, start=1):
    print(f"Key Clause {i}: {clause}")

Key Clause 1: Trí tuệ nhân tạo (AI) là mô phỏng quá trình trí tuệ của con người bởi máy móc, đặc biệt là các hệ thống máy tính.
Key Clause 2: Các quá trình này bao gồm học hỏi (tiếp thu thông tin và quy tắc để sử dụng thông tin đó), lập luận (sử dụng quy tắc để đạt đến kết luận gần đúng hoặc xác định) và tự sửa lỗi.
Key Clause 3: AI có thể được phân loại thành AI yếu và AI mạnh.
Key Clause 4: AI yếu, còn được gọi là AI hẹp, là một hệ thống AI được thiết kế và huấn luyện cho một nhiệm vụ cụ thể.
Key Clause 5: Trợ lý cá nhân ảo, chẳng hạn như Siri của Apple, là một dạng AI yếu.
Key Clause 6: AI mạnh, còn được gọi là trí tuệ nhân tạo tổng quát, là một hệ thống AI có khả năng tổng quát hóa như trí tuệ nhân tạo của con người.
Key Clause 7: Khi được đưa ra một nhiệm vụ không quen thuộc, một hệ thống AI mạnh có thể tìm ra giải pháp mà không cần sự can thiệp của con người.


In [30]:
from _2_1_pre_processing import data_preprocessing

In [31]:
train_data['sentence'][:20]

['slide giáo_trình đầy_đủ .',
 'nhiệt_tình giảng_dạy , gần_gũi sinh_viên .',
 'đi học đầy_đủ full_điểm chuyên .',
 'chưa áp_dụng công_nghệ_thông_tin thiết_bị hỗ_trợ giảng_dạy .',
 'thầy giảng bài hay , bài_tập ví_dụ lớp .',
 'giảng_viên đảm_bảo thời_gian lên_lớp , tích_cực trả_lời câu hỏi sinh_viên , thường_xuyên đặt câu hỏi sinh_viên .',
 'em nợ môn , em học ở học_kỳ kế_tiếp .',
 'thời_lượng học quá dài , đảm_bảo tiếp_thu hiệu_quả .',
 'nội_dung môn_học phần thiếu trọng_tâm , hầu_như chung_chung , khái_quát khiến sinh_viên khó nắm nội_dung môn_học .',
 'nói rõ hơn bằng cách trình_bày bảng thay_vì nhìn slide .',
 'thầy tận_tình đi dạy đúng giờ .',
 'dễ áp_lực .',
 'dạy thầy wzjwz đi nước_ngoài giữa_chừng , thầy wzjwz dạy thay .',
 'môn_học còn dễ nâng_độ khó .',
 'sinh_viên đi thăm_quan công_ty xem quy_mô cách làm_việc , giúp hiểu rõ hơn vê ngành mình học .',
 'hài_lòng về tất_cả .',
 'nhiệt_tình , vui_vẻ .',
 'thấy nhiệt_tình trong trao_đổi giúp_đỡ về học_tập .',
 'phong_cách giảng_bà

In [32]:
train_data = train_data.map(lambda example: {'sentence': data_preprocessing(example['sentence'])})

Map: 100%|██████████| 11426/11426 [00:06<00:00, 1706.08 examples/s]


In [33]:
train_data['sentence'][:20]

['bài trình chiếu giáotrình đầyđủ',
 'nhiệttình giảngdạy gầngũi sinhviên',
 'đi học đầyđủ fullđiểm chuyên',
 'chưa ápdụng côngnghệthôngtin thiếtbị hỗtrợ giảngdạy',
 'thầy giảng bài hay bàitập vídụ lớp',
 'giảngviên đảmbảo thờigian lênlớp tíchcực trảlời câu hỏi sinhviên thườngxuyên đặt câu hỏi sinhviên',
 'em nợ môn em học ở họckỳ kếtiếp',
 'thờilượng học quá dài đảmbảo tiếpthu hiệuquả',
 'nộidung mônhọc phần thiếu trọngtâm hầunhư chungchung kháiquát khiến sinhviên khó nắm nộidung mônhọc',
 'nói rõ hơn bằng cách trìnhbày bảng thayvì nhìn bài trình chiếu',
 'thầy tậntình đi dạy đúng giờ',
 'dễ áplực',
 'dạy thầy wzjwz đi nướcngoài giữachừng thầy wzjwz dạy thay',
 'mônhọc còn dễ nângđộ khó',
 'sinhviên đi thămquan côngty xem quymô cách làmviệc giúp hiểu rõ hơn về ngành mình học',
 'hàilòng về tấtcả',
 'nhiệttình vuivẻ',
 'thấy nhiệttình trong traođổi giúpđỡ về họctập',
 'phongcách giảngbài thầy gầngũi sinhviên',
 'giảngdạy kếthợp hoạtđộng giảitrí hiệuquả']